# A Estratégia

Tempo Gráfico de 1 minuto

Média Móvel de 20 período aritmetica

Condições

    Compra: Os 2 primeiros fechamentos de candle acima da média móvel, o 3° sera uma compra.

    Venda: Os 2 primeiros fechamentos de candle abaixo da média móvel, o 3° sera de venda.


Estratégia que utiliza tendencia para entradas.

Efetuar backtest com:

    Gale 1x
    Gale 2x


In [20]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import MetaTrader5 as mt5

from scipy.signal import argrelextrema

import pytz

In [21]:
login, password, server = open("credentials.txt").read().split()

periodo_media = 20

In [22]:
if not mt5.initialize(login = int(login), password = password, server = server):
    print("Erro !")

In [23]:
rates = mt5.copy_rates_range("EURUSD", mt5.TIMEFRAME_M5, datetime(2024, 2, 11, 0, 0, 0), datetime(2024, 2, 12, 3, 0,0))

mt5.shutdown()

True

In [24]:
rates_frame = pd.DataFrame(rates)

In [25]:
rates_frame["date_time"] = pd.to_datetime(rates_frame["time"], unit="s")

In [35]:
rates_frame["ma20"] = rates_frame["close"].rolling(window = periodo_media).mean()

In [37]:
rates_frame = rates_frame.sort_values(by = "time", ascending= True)

rates_frame.to_excel("rates_frame.xlsx")

In [ ]:
# Analisa o fechamento e a media anterior se são maiores e se o fechamento e a media posterior se são menores
np.where((rates_frame['close'].shift(1) < rates_frame['ma20'].shift(1)) & (rates_frame['close'] > rates_frame['ma20']))[0]

In [76]:
crossings = np.where((rates_frame['close'].shift(1) > rates_frame['ma20'].shift(1)) & (rates_frame['close'] < rates_frame['ma20']))[0]

In [89]:
# Encontrar o terceiro candle após cada cruzamento
markers_x = []
markers_y = []

for crossing in crossings:
    # Verificar se há pelo menos três candles após o cruzamento
        third_candle = rates_frame.iloc[crossing + 2]
        if third_candle['close'] < third_candle['open']:
            markers_x.append(third_candle['date_time'])
            markers_y.append(third_candle['close'])

In [90]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Criar o gráfico de candlestick
candlestick =  go.Candlestick(x=rates_frame['date_time'],
                            open=rates_frame['open'],
                            high=rates_frame['high'],
                            low=rates_frame['low'],
                            close=rates_frame['close'])

# Media movel
ma = go.Scatter(x=rates_frame['date_time'], y=rates_frame['mean'], mode='lines', line = go.scatter.Line(color = "DeepPink"), name='MA20')


# buy = go.Scatter(x=rates_frame[rates_frame['close'] > rates_frame['mean']]['date_time'],
#                      y=rates_frame[rates_frame['close'] > rates_frame['mean']]['high'],
#                      mode='markers',
#                      marker=dict(symbol='x', color='green', size=10),
#                      name='Above MA')



# Adicionar marcador para o terceiro candle após o cruzamento
marker = go.Scatter(x=markers_x, y=markers_y,
                    mode='markers', name='Third Candle After Crossing MA20', marker=dict(color='purple', size=10))


fig = make_subplots(rows = 2, cols = 1, shared_xaxes = True)


# Adicionando os gráficos ao subplot
fig.add_trace(candlestick, row=1, col=1)
fig.add_trace(ma, row=1, col=1)
fig.add_trace(marker)


# Atualizar layout
fig.update_layout(title='Candlestick (OHLC)',
                  xaxis_title='Data',
                  yaxis_title='Preço',
                  template = 'seaborn', 
                  xaxis_rangeslider_visible = False,
                  showlegend = True,
                  height = 900)

# Exibir o gráfico
fig.show()